In [1]:
import pandas as pd
import numpy as np

In [7]:
members = pd.read_csv("./data/HSall_members.csv")
members.tail()

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,bioguide_id,born,died,dim1,dim2,log_likelihood,geo_mean_probability,number_of_votes,number_of_errors,conditional
48799,115,Senate,40915,56,0,WV,100,NaN,NaN,"MANCHIN, Joe, III",M001183,1947.0,NaN,-0.064,0.458,-32.63383,0.82534,170.0,17.0,1.0
48800,115,Senate,29940,25,0,WI,100,NaN,NaN,"BALDWIN, Tammy",B001230,1962.0,NaN,-0.542,-0.255,-31.30662,0.83181,170.0,16.0,1.0
48801,115,Senate,41111,25,0,WI,200,NaN,NaN,"JOHNSON, Ron",J000293,1955.0,NaN,0.594,-0.328,-3.41540,0.97999,169.0,1.0,1.0
48802,115,Senate,40707,68,0,WY,200,NaN,NaN,"BARRASSO, John A.",B001261,1952.0,NaN,0.528,0.226,-2.64773,0.98446,169.0,1.0,1.0
48803,115,Senate,49706,68,0,WY,200,NaN,NaN,"ENZI, Michael B.",E000285,1944.0,NaN,0.537,0.177,-4.21116,0.97553,170.0,2.0,1.0


In [6]:
bills = pd.read_csv("./data/HSall_rollcalls.csv")
bills.tail()

/Users/gaoag/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,congress,chamber,rollnumber,date,session,clerk_rollnumber,mid_1,mid_2,spread_1,spread_2,log_likelihood,bill_number,vote_result,vote_desc,vote_question,dtl_desc
105249,115,Senate,183,2017-08-02,1.0,183.0,-0.675,0.575,-0.675,0.575,-0.145,PN681,Cloture Motion Agreed to,NaN,NaN,NaN
105250,115,Senate,184,2017-08-02,1.0,184.0,-0.675,0.575,-0.675,0.575,-0.145,PN681,Nomination Confirmed,NaN,NaN,NaN
105251,115,Senate,185,2017-08-03,1.0,185.0,0.000,0.000,0.000,0.000,0.000,HR2430,Cloture Motion Agreed to,NaN,NaN,NaN
105252,115,Senate,186,2017-08-03,1.0,186.0,-0.260,0.001,-0.260,0.001,-25.285,PN459,Nomination Confirmed,NaN,NaN,NaN
105253,115,Senate,187,2017-08-03,1.0,187.0,0.000,0.000,0.000,0.000,0.000,HR2430,Bill Passed,NaN,NaN,NaN


In [8]:
#this file is half a GB, so if you've got memory issues on your computer, don't do this! instead, load it in chunks.
votes = pd.read_csv("./data/HSall_votes.csv")
votes.tail()

,congress,chamber,rollnumber,icpsr,cast_code
24023853,115,Senate,187,49300,1
24023854,115,Senate,187,49308,1
24023855,115,Senate,187,49703,1
24023856,115,Senate,187,49706,1
24023857,115,Senate,187,94659,1


# Data organization 

Members gives the names of the members, their party codes, their specific congress # and chamber, and the affiliated ICPSR code. The party name itself is not included, but can be inferred (200=R, 100=D). 

Votes gives the specific congress/chamber, the ICPSR, the ‘cast code’ whatever that is... it wasn't defined in the source... and the rollnumber that the vote corresponds to

Bills connects a rollnumber to a specific bill, result, and bill number. Unfortunately, the bill name itself is usually not included.

Note: The rollnumber is not unique! the combo of chamber/congress/rollnumber is what’s unique and defines a specific vote.

# Data exploration/preprocessing

First, let's figure out what the 'cast code' means.

In [115]:
#what codes exist?
votes['cast_code'].unique()

array([6, 9, 1, 8, 2, 5, 7, 3, 4])

In [119]:
senate114votes = votes[(votes['congress'] == 114) & (votes['chamber'] == 'Senate')]

In [117]:
bills[(bills['congress'] == 114) & (bills['chamber'] == 'Senate') & (bills['rollnumber'] == 1)]

,congress,chamber,rollnumber,date,session,clerk_rollnumber,mid_1,mid_2,spread_1,spread_2,log_likelihood,bill_number,vote_result,vote_desc,vote_question,dtl_desc
104128,114,Senate,1,2015-01-08,1.0,1.0,0.392,0.266,0.392,0.266,-13.277,HR26,Amendment Rejected,NaN,NaN,NaN


In [118]:
senate114votes[(senate114votes['rollnumber'] == 1)]['cast_code'].value_counts()

6    66
1    31
9     3
Name: cast_code, dtype: int64

Alright, sick. By looking up the vote record online for H.R.26 rollcall 1, we can guess that 6 = Nay, 1 = Yea, and 9 = Not voting. But what's 8, 2, 5, 7, 3, and 4?...

In [126]:
senate114votes[(senate114votes['cast_code'] == 7)]

,congress,chamber,rollnumber,icpsr,cast_code
23785427,114,Senate,190,14503,7
23801984,114,Senate,355,40704,7
23803197,114,Senate,367,41104,7


In [130]:
bills[(bills['congress'] == 114) & (bills['chamber'] == 'Senate') & (bills['rollnumber'] == 190)]

,congress,chamber,rollnumber,date,session,clerk_rollnumber,mid_1,mid_2,spread_1,spread_2,log_likelihood,bill_number,vote_result,vote_desc,vote_question,dtl_desc
104317,114,Senate,190,2015-05-22,1.0,190.0,-0.254,-0.325,-0.254,-0.325,-21.809,HR1314,Amendment Rejected,NaN,NaN,NaN


After some inspection, turns out that the only other code appearing in our range of analysis (senate 104 to 114) is code 7. By looking up the vote records for HR1314, we can see that 7 is a 'present' vote, which has a different nuance than code 9 (not voting). Although it is functionally the same, 'present' reflects an intentional abstention or political strategy (https://www.conginst.org/2013/04/02/voting-present-as-a-legislative-tactic/) and is recorded as an official vote, and 'no vote' means the senator was off somewhere just... not doing their job, I guess. 

Now, let's inspect the data for senate #104.

In [133]:
ith_senate_votes = votes[(votes['congress'] == 104) & (votes['chamber'] == 'Senate')]
data_matrix = ith_senate_votes.pivot(index='rollnumber', columns='icpsr', values='cast_code')

In [134]:
data_matrix.head()

icpsr,1366,4812,9369,10513,10791,10808,10818,11044,11203,11204,...,49308,49309,49500,49501,49502,49503,49504,94659,95407,99909
rollnumber,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,...,1.0,-1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,0.0
2,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,0.0,...,-1.0,-1.0,-1.0,1.0,1.0,1.0,NaN,1.0,NaN,0.0
3,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,0.0,...,1.0,-1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,0.0
4,1.0,1.0,-1.0,-1.0,1.0,0.0,1.0,-1.0,-1.0,0.0,...,1.0,1.0,-1.0,-1.0,-1.0,-1.0,NaN,-1.0,NaN,0.0
5,-1.0,-1.0,1.0,1.0,1.0,0.0,-1.0,1.0,1.0,0.0,...,-1.0,-1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,0.0


There are 104 columns, but there are only 100 senators at any given time. The issue arises because senators come and go for whatever reasons; in this case, Sheila Frahm was a temporary replacement for Bob Dole and was voted out and replaced by Sam Brownback, and Ron Wyden was picked in a special election to replace Bob Packwood (who resigned after his sexual assaults were revealed). 

It is difficult to filter this automatically. Many of the senators who came and went voted a significant number of times (usually hundreds). For now, we will just filter out presidential tiebreakers.

In [135]:
#filter out presidential codes
data_matrix.drop(data_matrix.columns[data_matrix.columns > 99900], axis=1, inplace=True)

Now, we need to impute missing data. Let's fill in the blanks with the party averages. 

BUT WAIT. What exactly does an 'average' vote mean? The vote codes are 1, 6, 9, etc. What's a vote of '4.67433'??

We need to rework the 'vote codes' such that they mean something numerically. If, say, we let votes against a bill = -1, abstention = 0, and vote for = 1, then an average party vote of '0.6' would mean something like 'most of the party leaned towards voting for this bill'. This 'average' vector can be very useful later on for fun linear algebra things, but let's keep this focused on preprocessing.

In [136]:
votes['cast_code'].replace([1, 6, 7, 9], [1, -1, 0, 0], inplace=True)

In [137]:
#record what codes are rep/dem/other in the first place
rep_senators = set(members[members['party_code'] == 200]['icpsr'].values)
dem_senators = set(members[members['party_code'] == 100]['icpsr'].values)

In [94]:
#calc party averages
  
#step 1: filter out relevant cols. looks clumsy, but >100x faster than the built in 'pd.Intersection' method
rep_records = data_matrix[[col for col in data_matrix.columns if col in rep_senators]]
dem_records = data_matrix[[col for col in data_matrix.columns if col in dem_senators]]  

#step 2: for a missing entry, fill it with the average 'vote value' for that row; lil workaround since axis=1 is wack
rep_records_imputed = rep_records.T.fillna(rep_records.T.mean()).T
dem_records_imputed = dem_records.T.fillna(dem_records.T.mean()).T

#step 3: put the dem and rep records together for the final data matrix
rep_avg = rep_records_imputed.mean(axis=1)
dem_avg = dem_records_imputed.mean(axis=1)
rep_avg.name = 'rep_avg'
dem_avg.name = 'dem_avg'

final_data_matrix = rep_records_imputed.join(other=[dem_records_imputed, rep_avg, dem_avg])

Inspection of final_data_matrix checks out. let's do this for every other congressional chamber now.

In [138]:
for i in range(104, 115):
    ith_senate_votes = votes[(votes['congress'] == i) & (votes['chamber'] == 'Senate')]
    data_matrix = ith_senate_votes.pivot(index='rollnumber', columns='icpsr', values='cast_code')
    data_matrix.drop(data_matrix.columns[data_matrix.columns > 99900], axis=1, inplace=True)
    
    rep_records = data_matrix[[col for col in data_matrix.columns if col in rep_senators]]
    dem_records = data_matrix[[col for col in data_matrix.columns if col in dem_senators]]  

    rep_records_imputed = rep_records.T.fillna(rep_records.T.mean()).T
    dem_records_imputed = dem_records.T.fillna(dem_records.T.mean()).T

    rep_avg = rep_records_imputed.mean(axis=1)
    dem_avg = dem_records_imputed.mean(axis=1)
    rep_avg.name = 'rep_avg'
    dem_avg.name = 'dem_avg'
    
    final_data_matrix = rep_records_imputed.join(other=[dem_records_imputed, rep_avg, dem_avg])

    final_data_matrix.to_csv('{}_data_matrix.csv'.format(i))

In [140]:
bills['vote_result'].unique()

array([nan, 'Passed', 'Failed', 'Agreed to', 'Nomination Confirmed',
       'Amendment Agreed to', 'Joint Resolution Passed',
       'Motion to Table Agreed to', 'Nomination Rejected',
       'Motion Rejected', 'Bill Passed', 'Amendment Rejected',
       'Motion to Recommit Rejected', 'Motion Agreed to',
       'Resolution Agreed to', 'Joint Resolution Defeated',
       'Conference Report Agreed to', 'Concurrent Resolution Agreed to',
       'Motion to Table Failed',
       'Motion to Table Motion to Recommit Agreed to',
       'Cloture Motion Agreed to', 'Veto Sustained',
       'Decision of Chair Not Sustained', 'Motion to Reconsider Rejected',
       'Motion to Reconsider Agreed to', 'Point of Order Sustained',
       'Amendment Germane', 'Cloture Motion Rejected', 'Guilty',
       'Not Guilty', 'Resolution of Ratification Agreed to',
       'Motion for Attendance Agreed to', 'Motion to Postpone Rejected',
       'Amendment Not Germane', 'Foley', 'Resolution Rejected',
       'Point

As a final note: not all these votes make 'sense' to analyze. What we are presumably interested in is the 'Bill Passed' and 'Bill defeated' portions; however, there's also 'Passed' and 'Failed' and 'Agreed to', etc. Let's just make some more data versions with just Bill passed/defeated results.